<a href="https://colab.research.google.com/github/paul6598/AI_practice/blob/main/240413.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"joonhuk","key":"76f027317e936ce75c134a8cef33e532"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle


In [3]:
!kaggle datasets download -d lovishbansal123/diamond-dataset

100% 733k/733k [00:00<00:00, 847kB/s]
100% 733k/733k [00:00<00:00, 846kB/s]


In [4]:
!unzip -o /content/diamond-dataset.zip -d /content/data_set

Archive:  /content/diamond-dataset.zip
  inflating: /content/data_set/diamonds.csv  


In [34]:
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

In [20]:
data = pd.read_csv('/content/data_set/diamonds.csv')

In [21]:
print(data)

       Unnamed: 0  carat        cut color clarity  depth  table  price     x  \
0               1   0.23      Ideal     E     SI2   61.5   55.0    326  3.95   
1               2   0.21    Premium     E     SI1   59.8   61.0    326  3.89   
2               3   0.23       Good     E     VS1   56.9   65.0    327  4.05   
3               4   0.29    Premium     I     VS2   62.4   58.0    334  4.20   
4               5   0.31       Good     J     SI2   63.3   58.0    335  4.34   
...           ...    ...        ...   ...     ...    ...    ...    ...   ...   
53935       53936   0.72      Ideal     D     SI1   60.8   57.0   2757  5.75   
53936       53937   0.72       Good     D     SI1   63.1   55.0   2757  5.69   
53937       53938   0.70  Very Good     D     SI1   62.8   60.0   2757  5.66   
53938       53939   0.86    Premium     H     SI2   61.0   58.0   2757  6.15   
53939       53940   0.75      Ideal     D     SI2   62.2   55.0   2757  5.83   

          y     z  
0      3.98  2.43  

In [22]:
data.isnull().sum().sort_values() #결측치 확인

Unnamed: 0    0
carat         0
cut           0
color         0
clarity       0
depth         0
table         0
price         0
x             0
y             0
z             0
dtype: int64

In [23]:
data = pd.get_dummies(data)

In [24]:
data

,Unnamed: 0,carat,depth,table,price,x,y,z,cut_Fair,cut_Good,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,1,0.23,61.5,55.0,326,3.95,3.98,2.43,False,False,...,False,False,False,False,False,True,False,False,False,False
1,2,0.21,59.8,61.0,326,3.89,3.84,2.31,False,False,...,False,False,False,False,True,False,False,False,False,False
2,3,0.23,56.9,65.0,327,4.05,4.07,2.31,False,True,...,False,False,False,False,False,False,True,False,False,False
3,4,0.29,62.4,58.0,334,4.20,4.23,2.63,False,False,...,True,False,False,False,False,False,False,True,False,False
4,5,0.31,63.3,58.0,335,4.34,4.35,2.75,False,True,...,False,True,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53935,53936,0.72,60.8,57.0,2757,5.75,5.76,3.50,False,False,...,False,False,False,False,True,False,False,False,False,False
53936,53937,0.72,63.1,55.0,2757,5.69,5.75,3.61,False,True,...,False,False,False,False,True,False,False,False,False,False
53937,53938,0.70,62.8,60.0,2757,5.66,5.68,3.56,False,False,...,False,False,False,False,True,False,False,False,False,False
53938,53939,0.86,61.0,58.0,2757,6.15,6.12,3.74,False,False,...,False,False,False,False,False,True,False,False,False,False


In [28]:
data.iloc[:,8:] = data.iloc[:,8:].astype(int)

In [29]:
data

,Unnamed: 0,carat,depth,table,price,x,y,z,cut_Fair,cut_Good,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,1,0.23,61.5,55.0,326,3.95,3.98,2.43,0,0,...,0,0,0,0,0,1,0,0,0,0
1,2,0.21,59.8,61.0,326,3.89,3.84,2.31,0,0,...,0,0,0,0,1,0,0,0,0,0
2,3,0.23,56.9,65.0,327,4.05,4.07,2.31,0,1,...,0,0,0,0,0,0,1,0,0,0
3,4,0.29,62.4,58.0,334,4.20,4.23,2.63,0,0,...,1,0,0,0,0,0,0,1,0,0
4,5,0.31,63.3,58.0,335,4.34,4.35,2.75,0,1,...,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53935,53936,0.72,60.8,57.0,2757,5.75,5.76,3.50,0,0,...,0,0,0,0,1,0,0,0,0,0
53936,53937,0.72,63.1,55.0,2757,5.69,5.75,3.61,0,1,...,0,0,0,0,1,0,0,0,0,0
53937,53938,0.70,62.8,60.0,2757,5.66,5.68,3.56,0,0,...,0,0,0,0,1,0,0,0,0,0
53938,53939,0.86,61.0,58.0,2757,6.15,6.12,3.74,0,0,...,0,0,0,0,0,1,0,0,0,0


In [32]:
y = data.iloc[:,4]
x = pd.concat([data.iloc[:,1:4], data.iloc[:,5:]], axis=1)

In [36]:
from sklearn.model_selection import train_test_split

In [37]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2)

In [39]:

model = Sequential()
model.add(Dense(10, input_dim = x_train.shape[1] ,activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 10)                270       
                                                                 
 dense_5 (Dense)             (None, 256)               2816      
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_6 (Dense)             (None, 32)                8224      
                                                                 
 dropout_3 (Dropout)         (None, 32)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 33        
                                                                 
Total params: 11343 (44.31 KB)
Trainable params: 11343

In [ ]:
model.compile(loss='mean_squared_error', optimizer='adam',metrics=['accuracy'])

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(x_train, y_train,
                    epochs = 1000,
                    validation_split = 0.25,
                    batch_size=64,
                    verbose=1,
                    callbacks = [early_stopping_callback])

Epoch 1/1000
506/506 [==============================] - 6s 5ms/step - loss: 17735162.0000 - accuracy: 0.0000e+00 - val_loss: 13628567.0000 - val_accuracy: 0.0000e+00
Epoch 2/1000
506/506 [==============================] - 3s 5ms/step - loss: 10015552.0000 - accuracy: 0.0000e+00 - val_loss: 3166100.2500 - val_accuracy: 0.0000e+00
Epoch 3/1000
506/506 [==============================] - 3s 5ms/step - loss: 2815948.7500 - accuracy: 0.0000e+00 - val_loss: 1545285.6250 - val_accuracy: 0.0000e+00
Epoch 4/1000
506/506 [==============================] - 2s 4ms/step - loss: 2138031.5000 - accuracy: 0.0000e+00 - val_loss: 1150409.1250 - val_accuracy: 0.0000e+00
Epoch 5/1000
506/506 [==============================] - 2s 4ms/step - loss: 1826331.0000 - accuracy: 0.0000e+00 - val_loss: 1044476.3750 - val_accuracy: 0.0000e+00
Epoch 6/1000
506/506 [==============================] - 2s 4ms/step - loss: 1707542.5000 - accuracy: 0.0000e+00 - val_loss: 929261.5625 - val_accuracy: 0.0000e+00
Epoch 7/1000
5

In [ ]:
y_vloss = history.history['val_loss']
y_loss = history.history['loss']

x_len = np.arange(len(y_loss))

plt.plot(x_len, y_vloss, marker='.', c = 'red', label = 'value_loss')
plt.plot(x_len, y_loss, marker='.', c = 'blue', label = 'loss')

plt.legend(loc = 'upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
real_price = []
pred_price = []
x_num = []

n_iter = 0
y_prediction = model.predict(x_test).flatten()
for i in range(50):
  real = y_test[i]
  pred = y_prediction[i]
  real_price.append(real)
  pred_price.append(pred)
  print("실제가격: {:.2f}, 예상가격: {:.2f}".format(real, pred))
  n_iter += 1
  x_num.append(n_iter)

plt.plot(x_num, pred_price, label = 'prediction')
plt.plot(x_num, real_price, label = 'real')
plt.legend()
plt.show()